In [ ]:
import os
import shutil
# 소스 디렉토리 및 대상 디렉토리 경로 설정
source_root_label = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/VL'
source_root_image = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/VS'
target_dir_label = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'
target_dir_image = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/image'

# 대상 디렉토리 생성
os.makedirs(target_dir_label, exist_ok=True)
os.makedirs(target_dir_image, exist_ok=True)

# 모든 하위 폴더 탐색
for root_label, dirs_label, files_label in os.walk(source_root_label):
    for file_label in files_label:
        # JSON 파일일 경우만 복사
        if file_label.endswith('.json'):
            src_path_label = os.path.join(root_label, file_label)
            dst_path_label = os.path.join(target_dir_label, file_label)
            # 파일 복사
            shutil.copy(src_path_label, dst_path_label)

for root_image, dirs_image, files_image in os.walk(source_root_image):
    for file_image in files_image:
        # Jpg 파일일 경우만 복사
        if file_image.endswith('.jpg'):
            src_path_image = os.path.join(root_image, file_image)
            dst_path_image = os.path.join(target_dir_image, file_image)
            # 파일 복사
            shutil.copy(src_path_image, dst_path_image)


In [ ]:
import os
import json
import cv2
# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'

# 클래스명을 저장할 집합(set) 생성
class_names = set()

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # JSON 데이터에서 클래스명 추출하여 집합에 추가
            objects = data.get('objects', [])
            for obj in objects:
                class_name = obj.get('class_name', None)
                if class_name is not None:
                    class_names.add(class_name)

# 결과 출력
total_classes = len(class_names)
print("클래스 총 개수:", total_classes)
print("클래스명 목록:")
for class_name in sorted(class_names):
    print(class_name)

클래스 총 개수: 4
클래스명 목록:
invisible_signal
pedestrian_signal
unusual_signal
vehicular_signal


In [ ]:
import os
import json

# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'

# 유지할 클래스 이름 설정
keep_class_name = 'vehicular_signal'

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # JSON 데이터에서 유지할 클래스만 남기기
        objects = data.get('objects', [])
        filtered_objects = [obj for obj in objects if obj.get('class_name') == keep_class_name]
        data['objects'] = filtered_objects
        
        # JSON 파일 저장
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

print("필터링 작업이 완료되었습니다.")


필터링 작업이 완료되었습니다.


In [ ]:
import os
import json
import cv2
# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'

# 클래스명을 저장할 집합(set) 생성
class_names = set()

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # JSON 데이터에서 클래스명 추출하여 집합에 추가
            objects = data.get('objects', [])
            for obj in objects:
                class_name = obj.get('class_name', None)
                if class_name is not None:
                    class_names.add(class_name)

# 결과 출력
total_classes = len(class_names)
print("클래스 총 개수:", total_classes)
print("클래스명 목록:")
for class_name in sorted(class_names):
    print(class_name)

클래스 총 개수: 1
클래스명 목록:
vehicular_signal


바운딩 박스 좌상단 / 우하단 좌표
-> 꼭짓점 네개로 바꾸기~~~~~~~~~~~!!!

In [ ]:
import os
import json

label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'  # JSON 파일이 위치한 디렉토리 경로

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # JSON 데이터에서 객체 추출하여 바운딩 박스 좌표 변환
            objects = data.get('objects', [])
            for obj in objects:
                points = obj.get('data', [])
                if len(points) == 2:
                    pt1 = (int(points[0][0]), int(points[0][1]))
                    pt2 = (int(points[1][0]), int(points[1][1]))
                    
                    # 네 개의 꼭짓점 좌표
                    top_left = pt1
                    bottom_right = pt2
                    top_right = (pt2[0], pt1[1])
                    bottom_left = (pt1[0], pt2[1])
                    
                    # 네 개의 꼭짓점 좌표를 객체의 'data'에 다시 저장
                    obj['data'] = [top_left, top_right, bottom_right, bottom_left]

        # 변경된 데이터를 다시 JSON 파일로 저장
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)


In [ ]:
import os
import json
import cv2

# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'
image_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/image'
# 신호등 색깔을 저장할 집합(set) 생성
signal_colors = set()

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            data = json.load(f)
            
            # JSON 데이터에서 신호등 객체 추출하여 색깔 확인
            objects = data.get('objects', [])
            for obj in objects:
                class_name = obj.get('class_name', None)
                attribute = obj.get('attribute', {})
                signal_color = attribute.get('signal', None)
                if signal_color:
                    signal_colors.add(signal_color)
                    if signal_color == 'etc':
                        # JSON 데이터에서 네 개의 꼭짓점 좌표 가져오기
                        points = obj.get('data', [])
                        if len(points) == 4:  # 네 개의 점이 주어진 경우
                            # 이미지 파일 경로
                            img_filename = os.path.splitext(filename)[0] + '.jpg'
                            img_path = os.path.join(image_dir, img_filename)
                            
                            # 이미지 불러오기
                            img = cv2.imread(img_path)
                            
                            # 네 개의 꼭짓점 좌표 추출
                            pt1 = (points[0][0], points[0][1])
                            pt2 = (points[1][0], points[1][1])
                            pt3 = (points[2][0], points[2][1])
                            pt4 = (points[3][0], points[3][1])
                            
                            # 네모 그리기
                            cv2.rectangle(img, pt1, pt3, (0, 0, 255), 2)
                            
                            # "etc" 텍스트 추가
                            cv2.putText(img, 'etc', pt1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                            
                            # 이미지 출력
                            cv2.imshow('Traffic Light', img)
                            cv2.waitKey(0)
                            cv2.destroyAllWindows()
print(signal_colors)

넌 삭제다. etc

In [ ]:
import os
import json

# JSON 파일이 있는 디렉토리 경로 설정
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/label'
# 처리된 JSON 파일을 저장할 디렉토리 경로 설정
output_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/processed_label'

# 모든 JSON 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(label_dir, filename)
        
        # JSON 파일 열기
        with open(file_path, 'r') as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError:
                print(f"Error decoding JSON file: {file_path}")
                continue
            
            # JSON 데이터에서 "signal"이 "None"이거나 "etc"인 항목 제외하고 필터링
            if 'objects' in data:
                filtered_objects = [obj for obj in data['objects'] if obj.get('attribute', {}).get('signal') not in ['None', 'etc']]
                data['objects'] = filtered_objects
            else:
                print(f"Error: 'objects' key not found in JSON file: {file_path}")
                continue
            
        # 처리된 데이터를 새로운 JSON 파일로 저장
        output_path = os.path.join(output_dir, filename)
        with open(output_path, 'w') as f:
            json.dump(data, f, indent=4)



In [ ]:
import os

# 디렉토리 구조 생성 함수
def create_directories(base_dir='/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset'):
    dirs = [
        'images/train',
        'images/val',
        'images/test',
        'labels/train',
        'labels/val',
        'labels/test'
    ]
    for dir in dirs:
        os.makedirs(os.path.join(base_dir, dir), exist_ok=True)

create_directories()

yolo형식으로 바꾸기~~~~~!

In [ ]:
import json
import os
import shutil
from sklearn.model_selection import train_test_split

# 경로 설정
image_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/image'
label_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/processed_label'

base_output_dir = '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset'
output_image_dir = os.path.join(base_output_dir, 'images')
output_label_dir = os.path.join(base_output_dir, 'labels')
# 출력 디렉터리 생성
def create_directories(base_dir='/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset'):
    dirs = [
        'images/train',
        'images/val',
        'images/test',
        'labels/train',
        'labels/val',
        'labels/test'
    ]
    for dir in dirs:
        os.makedirs(os.path.join(base_dir, dir), exist_ok=True)

create_directories(base_output_dir)

# 클래스 딕셔너리
classes = {
    'red' : 0,
    'red and green arrow' :1 ,
    'green' : 2,
    'yellow' : 3,
    'green and green arrow' : 4,
    'green arrow' :5
}

# 바운딩 박스를 정규화하는 함수
def normalize_bbox(x_min, y_min, x_max, y_max, img_width, img_height):
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

# JSON 파일 목록을 가져옴
json_files = [f for f in os.listdir(label_dir) if f.endswith('.json')]

# 데이터를 train, val, test로 분할 (예: 70% train, 20% val, 10% test)
train_files, temp_files = train_test_split(json_files, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(temp_files, test_size=1/3, random_state=42)

# 파일 복사 및 변환 함수
def process_files(file_list, image_output_dir, label_output_dir):
    for json_file in file_list:
        with open(os.path.join(label_dir, json_file), 'r', encoding='utf-8') as f:
            data = json.load(f)
            objects = data.get('objects', [])

        
        image_filename = data.get('image_name')
        img_size = data.get('image_size')
        img_width, img_height = data.get('image_size', (None, None))
        
      
        if not image_filename or not img_size:
            image_filename = data.get('imagePath')
            img_width = data.get('imageWidth')
            img_height = data.get('imageHeight')
            continue
        
        
        # 이미지를 출력 디렉터리로 복사
        src_img_path = os.path.join(image_dir, image_filename)
        dst_img_path = os.path.join(image_output_dir, image_filename)
        if not os.path.exists(dst_img_path):
            shutil.copy(src_img_path, dst_img_path)
        
        # YOLO 형식 라벨 파일 생성
        label_filename = os.path.splitext(image_filename)[0] + '.txt'
        label_file_path = os.path.join(label_output_dir, label_filename)
        
        with open(label_file_path, 'w', encoding='utf-8') as label_file:
            for obj in objects:
                attribute = obj.get('attribute', {})
                signal = attribute.get('signal', None)
                
                # 클래스 ID 매핑
                if signal in classes:
                    class_id = classes[signal]
                else:
                    # 알 수 없는 신호일 경우 무시
                    continue
                
                points = obj.get('data', [])
                if len(points) == 4:
                    x_values = [point[0] for point in points]
                    y_values = [point[1] for point in points]
                    x_min = min(x_values)
                    x_max = max(x_values)
                    y_min = min(y_values)
                    y_max = max(y_values)
                    x_center, y_center, width, height = normalize_bbox(x_min, y_min, x_max, y_max, img_width, img_height)
                    label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
# 각 데이터셋에 대해 파일 처리
process_files(train_files, os.path.join(output_image_dir, 'train'), os.path.join(output_label_dir, 'train'))
process_files(val_files, os.path.join(output_image_dir, 'val'), os.path.join(output_label_dir, 'val'))
process_files(test_files, os.path.join(output_image_dir, 'test'), os.path.join(output_label_dir, 'test'))

In [ ]:
import yaml

data = {
    'train': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/images/train',
    'val': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/images/val',
    'test': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/images/test',
    'nc': 6,
    'names': ['red',
              'red and green arrow',
              'green',
              'yellow',
              'green and green arrow',
              'green arrow']
}

with open('/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [ ]:
from numba import cuda
device = cuda.get_current_device(); device.reset()

In [ ]:
from ultralytics import YOLO

# 모델 초기화
model = YOLO('yolov5s.pt')
# 하이퍼파라미터 설정
train_params = {
    'data': '/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/dataset.yaml',
    'epochs': 20,
    'batch': 4,
    'imgsz': 640,
    'lr0': 0.001,          # 학습률을 낮춤
    'momentum': 0.8,       # 모멘텀 값을 낮춤
    'weight_decay': 0.0001, # 가중치 감쇠를 낮춤
    'optimizer': 'Adam',
    'patience': None 
}

# 훈련 중간 체크포인트 저장 경로 설정
checkpoint_path = 'yolo_checkpoint.pt'

# 모든 에폭을 돌면서 모델을 훈련
for epoch in range(train_params['epochs']):
    # 훈련 진행
    results = model.train(**train_params)
    
    # 중간 체크포인트 저장
    model.save(checkpoint_path)

# 훈련이 끝나면 최종 모델을 저장
final_model_path = 'final_yolo_model.pt'
model.save(final_model_path)

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

New https://pypi.org/project/ultralytics/8.2.29 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.16 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5938MiB)
engine/trainer: task=detect, mode=train, model=yolov5s.pt, data=/home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/dataset.yaml, epochs=20, time=None, patience=None, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, mult

train: Scanning /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/train... 3463 images, 105 backgrounds, 0 corrupt: 100%|██████████| 3463/3463 [00:01<00:00, 2972.57it/s]


train: New cache created: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/train.cache


val: Scanning /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/val... 997 images, 36 backgrounds, 0 corrupt: 100%|██████████| 997/997 [00:00<00:00, 1027.54it/s]

val: New cache created: /home/min/dev_ws/machine_learning/project_3rd/data/traffic_detection/YOLO_dataset/labels/val.cache


...

       1/20       1.3G      1.092     0.6505     0.8267         16        640:   6%|▋         | 56/866 [00:06<01:37,  8.30it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c864d1e36d0>
Traceback (most recent call last):
  File "/home/min/venv/yolo/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/min/venv/yolo/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self

                   all        997       2984      0.807      0.625      0.718      0.474



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/20      1.22G      1.173     0.7367     0.8365          8        640:  17%|█▋        | 143/866 [00:13<01:06, 10.85it/s]


KeyboardInterrupt: 

In [1]:
import cv2
from ultralytics import YOLO

# 모델 경로
model_path = '/home/min/dev_ws/machine_learning/project_3rd/src/model_learning/traffic_light/traffic_best_ver2.pt'
model = YOLO(model_path)

# 동영상 파일 경로
video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
cap = cv2.VideoCapture(video_path)

desired_width = 640
desired_height = 640

# 클래스 ID와 색상 매핑
classes = {
    0: 'Red',
    1: 'Red and Green Arrow',
    2: 'Green',
    3: 'Yellow',
    4: 'Green and Green Arrow',
    5: 'Green Arrow'
}

while cap.isOpened():
    # 비디오에서 프레임을 읽음
    success, frame = cap.read()

    if success:
        # 프레임을 원하는 크기로 조정
        frame = cv2.resize(frame, (desired_width, desired_height))

        # 프레임에 YOLOv8 추적 수행, 프레임 간에 트랙 유지
        results = model.track(frame, persist=True)
        print(results[0].boxes)
        if len(results) > 0:
            # 결과에서 클래스 값 가져오기
            predicted_classes = []
            for detection in results[0].boxes.data:
                x1, y1, x2, y2 = detection[:4]
                confidence = detection[4]
                class_id = detection[5]
                predicted_classes.append(int(class_id))

            # 예측된 클래스 값을 이용해 원하는 정보 추출
            for class_id in predicted_classes:
                if class_id in classes:
                    print(f"Predicted color: {classes[class_id]}")
                else:
                    print("Predicted color: Unknown")

            # 결과를 시각화하여 프레임에 표시
            annotated_frame = results[0].plot()
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
            

        # 'q' 키가 눌렸는지 확인
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# 비디오 캡처 객체 해제 및 모든 창 닫기
cap.release()
cv2.destroyAllWindows()



0: 640x640 (no detections), 9.8ms
Speed: 2.9ms preprocess, 9.8ms inference, 430.6ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))



QFactoryLoader::QFactoryLoader() checking directory path "/home/min/venv/yolo/lib/python3.10/site-packages/cv2/qt/plugins" ...
QFactoryLoader::QFactoryLoader() checking directory path "/usr/bin" ...
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/411toppm"
"Failed to extract plugin meta data from '/usr/bin/411toppm'" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7z"
QElfParser: '/usr/bin/7z' is not an ELF object (file too small)
"'/usr/bin/7z' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7za"
QElfParser: '/usr/bin/7za' is not an ELF object (file too small)
"'/usr/bin/7za' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7zr"
QElfParser: '/usr/bin/7zr' is not an ELF object (file too small)
"'/usr/bin/7zr' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr

0: 640x640 (no detections), 15.3ms
Speed: 4.7ms preprocess, 15.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))

0: 640x640 (no detections), 7.3ms
Speed: 3.0ms preprocess, 7.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: t